## Optimization code for PV microgrid:

In [1]:
# # using JuMP, AmplNLWriter, MATLAB
# # m = Model(solver=AmplNLSolver("C:\Program Files\SCIPOptSuite 4.0.1\bin\scip.exe"))
# using JuMP, SCIP, MATLAB
# m = Model(solver=SCIPSolver())


In [2]:
### Model initialization:

using JuMP, Gurobi, MATLAB
# m = Model(solver=GurobiSolver(Presolve=2))
# m = Model(solver=GurobiSolver(Presolve=2,TimeLimit=300))
m = Model(solver=GurobiSolver(Presolve=1,TimeLimit=900))

# using JuMP, CPLEX, MATLAB
# # m = Model(solver=CplexSolver(CPX_PARAM_TILIM = 900))
# m = Model(solver=CplexSolver(CPX_PARAM_REPEATPRESOLVE = 3, CPX_PARAM_TILIM = 900))



Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is Gurobi

In [4]:
vars = read_matfile("../../Data/Generated Data/5 - Optimization/scenarios/test.mat")

# Parameters:

interval  = Int64(jvalue(vars["interval"]))  # Current Interval (in day, for which we are optimizing)

N_EMS   = Int64(jvalue(vars["N_EMS"]))   # num of remaining EMS   intervals (opt. executions)
N_intra = Int64(jvalue(vars["N_intra"])) # num of remaining Intra intervals (opt. resolution)
N_dies  = Int64(jvalue(vars["N_dies"]))  # num of diesel generators
N_scen  = Int64(jvalue(vars["N_scen"]))  # num of different scenations
N_firm  = Int64(jvalue(vars["N_firm"]))  # num of FIRM Margin Constraint Intervals

# Upper and Lower bounds:
P_bat_min = jvalue(vars["P_bat_min"])
P_bat_max = jvalue(vars["P_bat_max"])
eta_bat  = jvalue(vars["eta_bat"])
SOC_min  = jvalue(vars["SOC_min"])
SOC_max  = jvalue(vars["SOC_max"])
SOC_init = jvalue(vars["SOC_init"])
Cap_bat  = jvalue(vars["Cap_bat"])
P_dies_max = jvalue(vars["P_dies_max"])
P_dies_min = jvalue(vars["P_dies_min"])
P_PV_inst = jvalue(vars["P_PV_inst"])

delta_t = jvalue(vars["delta_t"]) # 1 min

# f_min = -100000
f_min = jvalue(vars["f_min"])
theta_i = jvalue(vars["theta_i"])
theta_d = jvalue(vars["theta_d"])
theta_b = jvalue(vars["theta_b"])
theta_p = jvalue(vars["theta_p"])

marge_dies = jvalue(vars["marge_dies"])

L_PV = jvalue(vars["L_PV"]) # generated PV scenarios
L_C  = jvalue(vars["L_C"]) # generated load scenarios

# Defining iterators:
ts  = 1:N_EMS
ps  = 1:N_intra
ds  = 1:N_dies
ss  = 1:N_scen
m1s = 1:N_firm
m2s = (N_firm+1):N_EMS


13:96

In [5]:
## Variable definitions:

@variables m begin
    
    # Battery variables:
    P_bat_min <= P_bat_set[t=ts,p=ps]  <= P_bat_max # Battery setpoint for time t,p (decision var.)
    0 <= P_bat_cha[t=ts,p=ps] <= P_bat_max
    0 <= P_bat_dis[t=ts,p=ps] <= P_bat_max
    X_bat[t=ts,p=ps], Bin                           # Operating mode indicator (decision var.)
                                                    # -> P_bat_set = 2*(X_bat-0.5)*P_bat_abs
    SOC_min <= SOC_bat[t=ts,p=ps] <= SOC_max        # Obtained SOC for time t,p (solution var.)
    
    # PV Plant:
    0 <= P_PV_set[t=ts,p=ps]  <= P_PV_inst          # PV setpoint variable (decision var.)
    0 <= P_PV[t=ts,p=ps,s=ss] <= L_PV[t,p,s]        # Actual PV generation (solution var.)
    
    # Diesel Generators:
    ON_dies[t=ts,d=ds], Bin                         # Dies. connectin setpoint (decision var.)
    0-5 <= P_dies[t=ts,p=ps,s=ss,d=ds] <= P_dies_max+5  # Actual dies. generation  (solution var.)
    # P_dies_min <= P_dies[t=ts,p=ps,s=ss,d=ds] <= P_dies_max # Actual dies. generation  (solution var.)
    
end

# print(m)


In [6]:
@constraints m begin
    
    # Constraints:
    #
    # B1 - Power balance equation
    # C1 - PV setpoint limitation
    # D1 - Bat. setpoint equation
    # D2 - Charging    mode const.
    # D3 - Discharging mode const.
    # E1 - FIRM    Diesel reserve margin (separated over scenarios)
    # E2 - RELAXED Diesel reserve margin (mean over scenarios)
    # F1 - 
    # F2 - SOC limitations
    # F3 - 
    # G1 - Min frequency limitation
    
    A1[t=ts,p=ps,s=ss,d=ds], P_dies[t,p,s,d] <= ON_dies[t,d]*(P_dies_max)
    A2[t=ts,p=ps,s=ss,d=ds], P_dies[t,p,s,d] >= ON_dies[t,d]*(P_dies_min)
    
    B1[t=ts,p=ps,s=ss], (P_PV[t,p,s] + sum(P_dies[t,p,s,d] for d=ds) + P_bat_set[t,p] - L_C[t,p,s]) == 0
    
    C1[t=ts,p=ps,s=ss],  P_PV[t,p,s] <= P_PV_set[t,p]
    
    D1[t=ts,p=ps], P_bat_set[t,p] == P_bat_cha[t,p] - P_bat_dis[t,p]
    D2[t=ts,p=ps], P_bat_cha[t,p] <= P_bat_max *    X_bat[t,p]
    D3[t=ts,p=ps], P_bat_dis[t,p] <= P_bat_max * (1-X_bat[t,p])
    
    # E1[t=ts,p=ps,s=ss],  sum(P_dies[t,p,s,d] for d=ds) <= sum((ON_dies[t,d]*P_dies_max) for d=ds) - marge_dies
    E1[t=m1s,p=ps,s=ss],  sum(P_dies[t,p,s,d] for d=ds) <= sum((ON_dies[t,d]*P_dies_max) for d=ds) - marge_dies
    E2[t=m2s,p=ps],  sum(P_dies[t,p,s,d] for d=ds,s=ss)/N_scen <= sum((ON_dies[t,d]*P_dies_max) for d=ds) - marge_dies
    
    F1[t=1,p=1],               SOC_bat[t,p] == SOC_init             - P_bat_set[t,p]*delta_t/Cap_bat
    F2[t=2:N_EMS,p=1],         SOC_bat[t,p] == SOC_bat[t-1,N_intra] - P_bat_set[t,p]*delta_t/Cap_bat
    F3[t=1:N_EMS,p=2:N_intra], SOC_bat[t,p] == SOC_bat[t,p-1]       - P_bat_set[t,p]*delta_t/Cap_bat

    G1[t=ts,p=ps], f_min <= theta_i + theta_d*sum(ON_dies[t,d] for d=ds) + theta_b*P_bat_set[t,p] + theta_p*P_PV_set[t,p]

end

# print(m)

In [7]:
# @objective(m, Max, sum(P_PV[t,p,s] for t=ts,p=ps,s=ss) - sum(N_scen*(1-eta_bat)*(P_bat_cha[t,p]+P_bat_dis[t,p]) for t=ts,p=ps) );

# Adding sum of ON_dies - to minimize their utilization:
# @objective(m, Max, sum((sum(P_PV[t,p,s] for s=ss)-N_scen*(1-eta_bat)*(P_bat_cha[t,p]+P_bat_dis[t,p])) for t=ts,p=ps) - sum(ON_dies[t,d] for t=ts,d=ds));

@objective(m, Max, sum((sum(P_PV[t,p,s] for s=ss)-N_scen*(1-eta_bat)*(P_bat_cha[t,p]+P_bat_dis[t,p])) for t=ts,p=ps));


In [8]:
status = solve(m)

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 607679 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [5e-09, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [4e-01, 1e+04]
Presolve removed 3686 rows and 4405 columns
Presolve time: 1.27s
Presolved: 192874 rows, 98987 columns, 595933 nonzeros
Variable types: 96405 continuous, 2582 integer (2582 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   27088    1.5607297e+07   1.226320e+06   0.000000e+00      5s
   59255    1.5336989e+07   7.688241e+05   0.000000e+00     10s
   88469    1.5117815e+07   5.356775e+05   0.000000e+00     15s
  113700    1.4896828e+07   4.037263e+05   0.000000e+00     20s
  140140    1.4533330e+07   1.787550e+05   0.000000e+00     25s
  168029    1.1478453e+07   4.337931e+05   0.000000e+00     30s
  187230  

:Optimal

In [33]:
write_matfile("../../Data/Generated Data/5 - Optimization/solutions/sol_test.mat"; 
    status    = string(status),
    interval  = interval,
    P_bat_set = getvalue(P_bat_set[:,:]), 
    SOC_bat   = getvalue(SOC_bat[:,:]),
    P_PV_set  = getvalue(P_PV_set[:,:]),
    P_PV      = getvalue(P_PV[:,:,:]),
    ON_dies   = getvalue(ON_dies[:,:]),
    P_dies    = getvalue(P_dies[:,:,:,:]))

In [32]:
# string(status)=="Optimal"